In [4]:
import pandas as pd
import pyodbc
import os
import gspread
from gspread_dataframe import get_as_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

from datetime import datetime
import numpy as np

In [5]:
print("Script Run Started")

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(r"C:/Users/Administrator/LIS/GCPL SAMT AI/python-459011-73acbdff4227.json", scope)
client = gspread.authorize(creds)


Script Run Started


In [6]:
# Open Sheet
sheet = client.open_by_key("1Xfd6VDgIWuOTym0ClmbCYXuDW7OtGeqMg37ukItoXdw") # Master Sheets ( Dimensions Sheet)
Data = sheet.worksheet("Data")
UserSheet = sheet.worksheet("User")


In [7]:
server = '157.119.230.120,4071'    
database = 'PowerBI_LISV2'      
username = 'sa'
password = 'Y0m@Sql.!.123'

# Connection string
conn_str = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""

In [8]:

query = """ 
WITH CTE AS 
(SELECT 
(TXT.UserName+'-'+ TXT.VISIT_DATE)AS UserVisitKey ,
TXT.Visit_Date,
TXT.UserName,
(TXT.VISIT_DATE +' '+TXT.CHEKINTIME) AS ChekinTime,
TXT.LISStoreCode,
(TXT.VISIT_DATE+'-'+TXT.LISStoreCode+'-'+TXT.UserName) AS StoreDateKey,
TXT.StoreName,TXT.Designation,
(TXT.VISIT_DATE +' '+TXT.CHECKOUTTIME ) AS Chekouttime,
TXT.FullName,
NoPermissionReason
FROM
(SELECT   FORMAT(CONVERT(DATETIME, SA.[VisitDate], 105), 'dd-MM-yyyy') AS Visit_Date,
SA.LISStoreCode,SA.StoreName, SA.UserName,SA.Designation,SA.FullName,
CONVERT(varchar, CAST([CheckIn-Time] AS TIME), 8) AS CHEKINTIME,
CONVERT(varchar, CAST([CheckOut-Time] AS TIME), 8) AS CHECKOUTTIME,
NoPermissionReason
FROM [PowerBI_LISV2].[DaburPharmaManagement].[ISPStoreAttendance_NormalExport] AS SA
LEFT JOIN [PowerBI_LISV2].[DaburPharmaManagement].UserMaster AS UM ON UM.UserName = SA.UserName
WHERE  [Month] = MONTH(GETDATE()-1) AND [Year] = YEAR(GETDATE()-1)
AND UM.Status =  'True' and  SA.UserName not like '%Star%'
)AS TXT
)
SELECT * FROM CTE;
"""

UserDataQuery ="""
SELECT 
DISTINCT
UM.EmployeeID,UM.UserName, 
UM.FullName,
UM.Region, UM.State,UM.City,UM.Status, UH.Supervisor,UM2.FullName as SupervisorName
from [PowerBI_LISV2].[DaburPharmaManagement].UserMaster AS UM 
LEFT JOIN [PowerBI_LISV2].[DaburPharmaManagement].UserHierarchyListReport AS UH ON UH.ISP = UM.UserName
LEFT JOIN [PowerBI_LISV2].[DaburPharmaManagement].UserMaster AS UM2 ON UH.Supervisor = UM2.UserName
WHERE UM.Status = 'True'
and  UM.UserName not like '%Star%' and  UM.UserName not like '%Test%' and  UM.UserName not like '%TEST%';

""" 

In [9]:
conn = pyodbc.connect(conn_str)
df = pd.read_sql(query, conn)
UserData = pd.read_sql(UserDataQuery, conn)

UserData2=UserData.drop(columns = ['FullName'])

C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_19652\3593879617.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\Administrator\AppData\Local\Temp\3\ipykernel_19652\3593879617.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  UserData = pd.read_sql(UserDataQuery, conn)


In [10]:
print("data extracted....")

data extracted....


In [11]:
loginlogout = pd.merge(df,UserData2, how = 'left',on = 'UserName')

In [12]:
dfy= loginlogout[(loginlogout['UserName']== 'DBR-ISP114') & (loginlogout['Visit_Date']== '09-06-2025') & (loginlogout['LISStoreCode']=='DBRPH-0022007588DEL0DL00296961')]
# dfx = dfx.drop_duplicates()
dfy.head(10)

UserVisitKey  Visit_Date    UserName           ChekinTime  \
8831  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 14:08:42   

                        LISStoreCode  \
8831  DBRPH-0022007588DEL0DL00296961   

                                           StoreDateKey        StoreName  \
8831  09-06-2025-DBRPH-0022007588DEL0DL00296961-DBR-...  SAHU SUPER MART   

       Designation          Chekouttime   FullName NoPermissionReason  \
8831  Merchandiser  09-06-2025 14:08:42  Dhirender        Stock Issue   

       EmployeeID Region  State   City Status   Supervisor SupervisorName  
8831  YM202101947  North  Delhi  Delhi   True  DBRPH-SUP-1  Gautam Mourya

In [13]:
loginlogout['ChekinRank'] = loginlogout.groupby(['StoreDateKey'])['ChekinTime'].rank(method='dense', ascending=True).astype(int)
loginlogout2= loginlogout[loginlogout['ChekinRank']==1]

In [14]:
loginlogout2=loginlogout2.drop(columns = ['ChekinRank'])

In [15]:
loginlogout2 = loginlogout2.drop_duplicates()

In [16]:
loginlogout2.at[0, 'LastUpdated'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [17]:
print("csv updated....")
loginlogout2.to_csv(r'C:\Users\Administrator\LIS\Dabur\daburloginlogout.csv', index=False)

loginlogout2 = loginlogout2.drop_duplicates()
#Sending df on sheet

csv updated....


In [18]:
dfy= loginlogout2[(loginlogout2['UserName']== 'DBR-ISP114') & (loginlogout2['Visit_Date']== '09-06-2025') & (loginlogout2['LISStoreCode']=='DBRPH-0022007588DEL0DL00296961')]
# dfx = dfx.drop_duplicates()
dfy.head(10)

UserVisitKey  Visit_Date    UserName           ChekinTime  \
8831  DBR-ISP114-09-06-2025  09-06-2025  DBR-ISP114  09-06-2025 14:08:42   

                        LISStoreCode  \
8831  DBRPH-0022007588DEL0DL00296961   

                                           StoreDateKey        StoreName  \
8831  09-06-2025-DBRPH-0022007588DEL0DL00296961-DBR-...  SAHU SUPER MART   

       Designation          Chekouttime   FullName NoPermissionReason  \
8831  Merchandiser  09-06-2025 14:08:42  Dhirender        Stock Issue   

       EmployeeID Region  State   City Status   Supervisor SupervisorName  \
8831  YM202101947  North  Delhi  Delhi   True  DBRPH-SUP-1  Gautam Mourya   

     LastUpdated  
8831         NaN